# Get SDSS QSO data 

The data is located at https://faculty.washington.edu/ivezic/macleod/qso_dr7/Southern.html 


In [ ]:
import os
url = "https://faculty.washington.edu/ivezic/macleod/qso_dr7/QSO_S82.tar.gz"
fpath = os.path.join(os.getcwd(), "QSO_S82.tar.gz")
if not os.path.exists(fpath):
    os.system(f'wget {url}')  

In [ ]:
! tar xzf QSO_S82.tar.gz

We confirm that all light curves are  there by testing that the number of files in the directory equals to the expected number of quasars:

In [4]:
assert len(os.listdir("QSO_S82")) == 9258

# Get PS1 QSO data

One way is to query the user interface of the MAST archive given the ra,dec positions of  changing-look quasar  candidates. This is a subset of the 9258 quasars above, selected given their variability properties as described in Suberlak+2021. The table of candidates can be downloaded from https://iopscience.iop.org/0004-637X/907/2/96/suppdata/apjabc698t5_ascii.txt?doi=10.3847/1538-4357/abc698. I copy-paste the text into a text file (in the github repo) called "CLQSO_candidates.txt"

In [1]:
from astropy.table import Table
colnames = ['dbID', 'SDSSJID', 'ra', 'dec',	'Redshift', 'log10_Lbol', 'log10_MBH', 
            'f_Edd','Delta(mag)',
            'Delta(sigma_G)', 'MedianPS1']
clqso = Table.read("CLQSO_candidates.txt", format='ascii',
                   names=colnames)

In [2]:
len(clqso)

40

The MAST archive expects a csv file with the following structure:
    
    target     ra   dec 
    
    


In [12]:
# so we make one :
clqso[['SDSS_NAME','RA','DEC']].write('CLQSO_cans_ra_dec.csv', format='csv',
                                names=('target','ra','dec'),
                                overwrite=True)


On  https://catalogs.mast.stsci.edu/panstarrs/ we select  "Crossmatch a List of Targets " option. I save the table as CSV file CLQSO_candidates_PS1_DR2.csv:

In [13]:
ps1 = Table.read('CLQSO_candidates_PS1_DR2.csv', format='csv')

From https://outerspace.stsci.edu/display/PANSTARRS/PS1+Detection+table+fields I find that units of `psfFLux` are Janskys.

In [ ]:
import functions as func
ps1['mag'] = func.flux2ab(ps1['psfFlux'])
ps1['magerr'] = func.flux2absigma(ps1['psfFlux'], ps1['psfFluxErr'])

# Get ZTF QSO data 

The ZTF DR7 data https://www.ztf.caltech.edu/ztf-public-releases.html  can be downloaded using the `wget`-based function written by Paula Sanchez-Saez here https://github.com/LSST-sersag/dle/blob/main/activities/workshop/T2_Photometric_Reverberation_Mapping/Tutorial%202_PhotoRM_IJ.ipynb . I modify it to do just the downloading, because I prefer to read the files with astropy:

In [ ]:
# check whether a file already exists...
dr_path = os.path.join(os.getcwd(), 'ZTFDR17')
if not os.path.exists(dr_path):
    os.makedirs(dr_path)
for row in clqso:
    sdss_name = row['SDSS_NAME']
    jname = f'J{sdss_name}'
    ra = row['RA']
    dec = row['DEC']
    for f in 'gri':
        file_name = f'{jname}_ZTF_{f}.csv'
        file_path = os.path.join(dr_path, file_name)
        # only query if it doesn't 
        if not os.path.exists(file_path):
            func.get_ZTF_DR_lc(name=jname, 
                          ra=ra, 
                          dec=dec, 
                          band=f, 
                          dr_path=dr_path)
